In [1]:
import setup
setup.init_django()

In [2]:
from market import services as market_services

In [3]:
def get_stock_indicators(ticker = "X:BTCUSD", days=30):
    queryset = market_services.get_stock_quotes_queryset(ticker, days=days)
    if queryset.count() == 0:
        raise Exception(f"Data for {ticker} not found")
    averages = market_services.get_daily_moving_averages(ticker)
    price_target = market_services.get_price_target(ticker, days=days, queryset=queryset)
    volume_trend_daily = market_services.get_volume_trend_daily(ticker, days=days, queryset=queryset)
    rsi_data = market_services.calculate_rsi(ticker)
    signals = []

    if averages.get('ma_5') > averages.get('ma_20'):
        signals.append(1)
    else:
        signals.append(-1)

    if averages.get('ma_20') > averages.get('ma_50'):
        signals.append(1)
    else:
        signals.append(-1)

    if averages.get('ma_50') > averages.get('ma_100'):
        signals.append(1)
    else:
        signals.append(-1)

    if averages.get('ma_100') > averages.get('ma_200'):
        signals.append(1)
    else:
        signals.append(-1)

    if price_target.get('conservative_target') > price_target.get('current_price'):
        signals.append(1)
    else:
        signals.append(-1)

    if volume_trend_daily.get("volume_change_percent_daily") > 20:
        signals.append(1)
    elif volume_trend_daily.get("volume_change_percent_daily") < -20:
        signals.append(-1)
    else:
        signals.append(0)

    rsi = rsi_data.get('rsi')
    if rsi > 70:
        signals.append(-1)  # Overbought
    elif rsi < 30:
        signals.append(1) # Oversold
    else:
        signals.append(0)

    return {
        "days": days,
        "score": sum(signals),
        "ticker": ticker,
        "indicators": {
            **averages,
            **price_target,
            **volume_trend_daily,
            **rsi_data,
        }
        
    }

In [4]:
results = get_stock_indicators(ticker="X:BTCUSD", days=365)
score = results.get("score")
results

{'days': 365,
 'score': 2,
 'ticker': 'X:BTCUSD',
 'indicators': {'ma_5': 115198.38,
  'ma_20': 116809.8025,
  'ma_50': 112911.1256,
  'ma_100': 108609.1794,
  'ma_200': 99546.7003,
  'current_price': 117515.49,
  'conservative_target': 144523.29874,
  'aggressive_target': 161208.75126000002,
  'average_price': 89498.13543715847,
  'period_high': 123231.07,
  'period_low': 52530.0,
  'avg_volume_daily': 12133.646692328766,
  'latest_volume_daily': 8927.4669,
  'volume_change_percent_daily': -26.42387629727017,
  'rsi': 57.8118,
  'avg_gain': 12.3408,
  'avg_loss': 9.0057,
  'period': 14}}

In [5]:
import json
from decouple import config

In [6]:
results_as_json = json.dumps(results)
results_as_json

'{"days": 365, "score": 2, "ticker": "X:BTCUSD", "indicators": {"ma_5": 115198.38, "ma_20": 116809.8025, "ma_50": 112911.1256, "ma_100": 108609.1794, "ma_200": 99546.7003, "current_price": 117515.49, "conservative_target": 144523.29874, "aggressive_target": 161208.75126000002, "average_price": 89498.13543715847, "period_high": 123231.07, "period_low": 52530.0, "avg_volume_daily": 12133.646692328766, "latest_volume_daily": 8927.4669, "volume_change_percent_daily": -26.42387629727017, "rsi": 57.8118, "avg_gain": 12.3408, "avg_loss": 9.0057, "period": 14}}'

In [7]:
OPENAI_API_KEY=config("OPENAI_API_KEY", default=None)

In [8]:
assert OPENAI_API_KEY is not None

In [9]:

from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

In [10]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are an expert an analyzing stocks and respond in JSON data"},
        {"role": "user", "content": f"Considering these results {results_as_json}, provide a recommendation"}
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "recommendation",
            "schema": {
                "type": "object",
                "properties": {
                    "buy": {
                        "description": "Recommend to buy stock",
                        "type": "boolean"
                    },
                    "sell": {
                        "description": "Recommend to sell stock",
                        "type": "boolean"
                    },
                    "hold": {
                        "description": "Recommend to hold stock",
                        "type": "boolean"
                    },
                    "explanation": {
                        "description": "Explanation of reasoning in 1 or 2 sentences",
                        "type": "string"
                    },
                    "additionalProperties": False
                }
            }
        }
    }
)

In [11]:
result = json.loads(response.choices[0].message.content)
result

{'buy': True,
 'sell': False,
 'hold': False,
 'explanation': 'The current price of BTCUSD is above the moving averages, indicating bullish momentum, and with a conservative target significantly higher than the current price, it suggests strong potential for price appreciation.'}